In [37]:
from torchqc.states import QuantumState
import numpy as np
import torch

In [38]:
qubit_state = QuantumState.basis(2)[0]
print(qubit_state.state_tensor)
print(qubit_state.state_tensor.shape)

tensor([[1.+0.j],
        [0.+0.j]], dtype=torch.complex128)
torch.Size([2, 1])


In [39]:
basis_states = QuantumState.basis(2)

arbitrary_state = (basis_states[0] + basis_states[1]).normalize()

inner_product_value = arbitrary_state.inner_product(basis_states[0])
print(inner_product_value)

tensor([[0.7071+0.j]], dtype=torch.complex128)


In [40]:
from torchqc.operators import Operator, DynamicOperator

# define operators
sigmax = Operator(dims=2, matrix=torch.from_numpy(
    np.array([[0.j, 1], [1, 0]])))
sigmay = Operator(dims=2, matrix=torch.from_numpy(
    np.array([[0., -1.j], [1.j, 0]])))

# operators composition
composition = sigmax * sigmay
print("Composition = ", composition.matrix)

# operators addition
addition = sigmax + sigmay
print("Addition = ", addition.matrix)

# oprator acts on a state vector
zero_state = QuantumState.basis(2)[0]
one_state = sigmax.mul(zero_state)
print("New state = ", one_state.state_tensor)

Composition =  tensor([[0.+1.j, 0.+0.j],
        [0.+0.j, 0.-1.j]], dtype=torch.complex128)
Addition =  tensor([[0.+0.j, 1.-1.j],
        [1.+1.j, 0.+0.j]], dtype=torch.complex128)
New state =  tensor([[0.+0.j],
        [1.+0.j]], dtype=torch.complex128)


In [ ]:
from torchqc.common_matrices import sigmaX, sigmaZ

time = np.arange(0, 1, 0.2)

def Ht(t, args = []):
    index = int(np.where(time == t.numpy()[0])[0][0])

    if index % 2 == 0:
        return sigmaX().matrix
    else:
        return sigmaZ().matrix
    
Ht = DynamicOperator(dims=2, Ht=Ht, time=time)

tensor([[[ 0.+0.j,  1.+0.j],
         [ 1.+0.j,  0.+0.j]],

        [[ 1.+0.j,  0.+0.j],
         [ 0.+0.j, -1.+0.j]],

        [[ 0.+0.j,  1.+0.j],
         [ 1.+0.j,  0.+0.j]],

        [[ 1.+0.j,  0.+0.j],
         [ 0.+0.j, -1.+0.j]],

        [[ 0.+0.j,  1.+0.j],
         [ 1.+0.j,  0.+0.j]]], dtype=torch.complex128)


In [42]:
from torchqc.states import QuantumState
from torchqc.common_functions import get_density_matrix

zero_state = QuantumState.basis(2)[0]
rho = get_density_matrix(zero_state)
print(rho.matrix)

tensor([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]], dtype=torch.complex128)


In [43]:
from torchqc.tensor_product import tensor_product_ops, \
        tensor_product_states, partial_trace

basis_states = QuantumState.basis(2)
psi_1 = (basis_states[0] + basis_states[1]).normalize()
psi_2 = (basis_states[0] - basis_states[1]).normalize()

psi = tensor_product_states(psi_1, psi_2)
print("Composite state = ", psi.state_tensor)
print("Product dims = ", psi.product_dims)

rho_1 = get_density_matrix(psi_1)
rho_2 = get_density_matrix(psi_2)

print("Density matrix 1 = ", rho_1.matrix)
print("Density matrix 2 = ", rho_2.matrix)

rho = tensor_product_ops(rho_1, rho_2)
print("Composite density matrix = ", rho.matrix)
print("Product dims = ", rho.product_dims)

print("\nTracing out system 2:")
rho_1 = partial_trace(rho, [1])
print("Density matrix 1 (by partial trace): ", rho_1.matrix)

print("Tracing out system 1:")
rho_2 = partial_trace(rho, [0])
print("Density matrix 2 (by partial trace): ", rho_2.matrix)

Composite state =  tensor([[ 0.5000+0.j],
        [-0.5000+0.j],
        [ 0.5000+0.j],
        [-0.5000+0.j]], dtype=torch.complex128)
Product dims =  [2, 2]
Density matrix 1 =  tensor([[0.5000+0.j, 0.5000+0.j],
        [0.5000+0.j, 0.5000+0.j]], dtype=torch.complex128)
Density matrix 2 =  tensor([[ 0.5000+0.j, -0.5000+0.j],
        [-0.5000+0.j,  0.5000-0.j]], dtype=torch.complex128)
Composite density matrix =  tensor([[ 0.2500+0.j, -0.2500+0.j,  0.2500+0.j, -0.2500+0.j],
        [-0.2500+0.j,  0.2500+0.j, -0.2500+0.j,  0.2500+0.j],
        [ 0.2500+0.j, -0.2500+0.j,  0.2500+0.j, -0.2500+0.j],
        [-0.2500+0.j,  0.2500+0.j, -0.2500+0.j,  0.2500+0.j]],
       dtype=torch.complex128)
Product dims =  [2, 2]

Tracing out system 2:
Density matrix 1 (by partial trace):  tensor([[0.5000+0.j, 0.5000+0.j],
        [0.5000+0.j, 0.5000+0.j]], dtype=torch.complex128)
Tracing out system 1:
Density matrix 2 (by partial trace):  tensor([[ 0.5000+0.j, -0.5000+0.j],
        [-0.5000+0.j,  0.5000+

In [44]:
from torchqc.common_functions import bell_states, concurrence

states = bell_states()
rho = get_density_matrix(states[0])
concurrence(rho)

tensor(1.0000, dtype=torch.float64)

In [45]:
def QMCT(initial_state: QuantumState, hamiltonian: Operator, time: np.ndarray, Dt: float, jump_operators: list = [], damp_rates = []) -> list:
    r"""
    QMCT(initial_state: QuantumState, hamiltonian: Operator, time: np.array, Dt: float, jump_operators: list = [], damp_rates = []) -> list
    
    Returns a list of states as the qunatum system evolves in time
    
    The shapes of the :attr:`initial_state` and the :attr:`hamiltonian` tensor need
    to match.
    
    Args:
        initial_state (QuantumState): the initial quantum state.
        hamiltonian  (Operator): hamiltonian matrix or matrices
        time (np.ndarray): time in discrete time steps
        Dt (float): time step duration
        jump_operators: list[Operator]
            list of jump operators
        damp_rates: list[float]
            list of jump rates
    """


    Heff = hamiltonian

    for rate, jump_op in zip(damp_rates, jump_operators):
        Heff -= (1j / 2) * rate * jump_op.dagger() * jump_op

    time_tensor = torch.from_numpy(time).reshape(len(time), 1)
    time_tensor.requires_grad_(True)

    current_state = initial_state
    states = [initial_state]

    for i in range(len(time_tensor) - 1):
        Ht = Heff
        
        current_state_tensor = torch.matmul(torch.linalg.matrix_exp(-1j * Ht * Dt), current_state.state_tensor)
        
        current_state = QuantumState(current_state.dims, current_state_tensor)
        states.append(current_state)

    # if initial state is product state
    if initial_state.is_product:
        product_dims = initial_state.product_dims

        for state in states:
            state.is_product = True
            state.product_dims = product_dims

    return states